In [3]:
import time
import json
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np 


In [45]:
#page 似乎在30页之后都没有了，我们从第二页开始，所以page统一不变，是28
pages = 28


occupationName="Mental health counselor"

dic1 = {"data analyst":"-jobs-SRCH_KO0,12",
      "data scientist":"-jobs-SRCH_KO0,14",
       "data engineer": "-jobs-SRCH_KO0,13",
        "Consultant": "-jobs-SRCH_KO0,10",
        "Accountant":"-jobs-SRCH_KO0,10",
        "Graphic Designer":"-jobs-SRCH_KO0,16",
        "UX Designer":"-jobs-SRCH_KO0,11",
        "UI Designer":"-jobs-SRCH_KO0,11",
        "Art Director":"-jobs-SRCH_KO0,12",
        "Mental health counselor":"-jobs-SRCH_KO0,23"
       }

code = dic1[occupationName]


In [46]:
#get url list
origin="https://www.glassdoor.com/Job/"
occu=occupationName.lower()
occu=occu.replace(' ','-')
origin=origin+occu+code
urls=[origin +"_IP" + str(i+2)+ ".htm" for i in range(pages)]
urls

['https://www.glassdoor.com/Job/mental-health-counselor-jobs-SRCH_KO0,23_IP2.htm',
 'https://www.glassdoor.com/Job/mental-health-counselor-jobs-SRCH_KO0,23_IP3.htm',
 'https://www.glassdoor.com/Job/mental-health-counselor-jobs-SRCH_KO0,23_IP4.htm',
 'https://www.glassdoor.com/Job/mental-health-counselor-jobs-SRCH_KO0,23_IP5.htm',
 'https://www.glassdoor.com/Job/mental-health-counselor-jobs-SRCH_KO0,23_IP6.htm',
 'https://www.glassdoor.com/Job/mental-health-counselor-jobs-SRCH_KO0,23_IP7.htm',
 'https://www.glassdoor.com/Job/mental-health-counselor-jobs-SRCH_KO0,23_IP8.htm',
 'https://www.glassdoor.com/Job/mental-health-counselor-jobs-SRCH_KO0,23_IP9.htm',
 'https://www.glassdoor.com/Job/mental-health-counselor-jobs-SRCH_KO0,23_IP10.htm',
 'https://www.glassdoor.com/Job/mental-health-counselor-jobs-SRCH_KO0,23_IP11.htm',
 'https://www.glassdoor.com/Job/mental-health-counselor-jobs-SRCH_KO0,23_IP12.htm',
 'https://www.glassdoor.com/Job/mental-health-counselor-jobs-SRCH_KO0,23_IP13.htm',


In [47]:
#网上查自己的user agent 改下再运行
headers = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15"}
df = pd.DataFrame(columns = ['title','loc'])
index = 0
for i in range(pages):
    html = requests.get(urls[i], headers = headers)
    if(not html.ok):
        print(str(i) +'times try is not respond')
    soup = BeautifulSoup(html.content, "html.parser")
    #html.content
    job_list_1 = soup.find_all('li',attrs = {"class":'jl'})
    job_list_2 = soup.find_all('li',attrs = {"class":'jl selected'})
    for j in range(len(job_list_1)):
        k = job_list_1[j]
        loc = k['data-job-loc']
        title = k['data-normalize-job-title']
        df.loc[index] = [title,loc]
        index +=1
        
    for j in range(len(job_list_2)):
        k = job_list_1[j]
        loc = k['data-job-loc']
        title = k['data-normalize-job-title']
        df.loc[index] = [title,loc]
        index +=1
        
        
    #time.sleep(3)
    
    
df.head()

,title,loc
0,mental health counselor,"Royersford, PA"
1,mental health counselor,"Martinsville, IN"
2,mental health therapist,"Lake Villa, IL"
3,residential counselor,"Fall River, MA"
4,school counselor,"Tucson, AZ"


In [48]:
df['loc'] = [i[-2:] for i in df['loc'].values]
df.head()

,title,loc
0,mental health counselor,PA
1,mental health counselor,IN
2,mental health therapist,IL
3,residential counselor,MA
4,school counselor,AZ


In [49]:
freq = df['loc'].value_counts()
name = freq.index.values
count = freq.values
df = pd.DataFrame(data ={'state': name,'count':count})
df.head()

,state,count
0,CA,181
1,NY,58
2,WA,58
3,PA,42
4,IL,42


In [50]:
#write to csv
occu=occupationName.lower()
fname = occu.replace(' ','_')+'_locations.csv'
df.to_csv(fname, index = False)

In [51]:
pd.read_csv(fname).head()

,state,count
0,CA,181
1,NY,58
2,WA,58
3,PA,42
4,IL,42
